In [1]:
import numpy as np
import cv2
import datetime
#<add code to run ffmpeg command via ssh>
cap0 = cv2.VideoCapture("28_01_24_Arve_D.MP4")
cap1 = cv2.VideoCapture("28_01_24_Arve_G.MP4")
fps = cap0.get(cv2.CAP_PROP_FPS)
while cap0.isOpened() and cap1.isOpened():
    # Capture frame-by-frame
    ret0, frame0 = cap0.read()
    ret1, frame1 = cap1.read()
    if ret0== True and ret1 == True:
        cv2.imshow('video', frame0)
        cv2.imshow('video', frame1)
        if cv2.waitKey(int(1000/fps)) == ord("q"):
            break
    else:
        break
cap0.release()
cap1.release()

cv2.destroyAllWindows()